In [29]:
#importing bunch of libraries that would be needed in future
import pandas as pd
import numpy as np
import gc,os
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
import pickle
import lightgbm as lgb
from sklearn.metrics import roc_auc_score,precision_score,recall_score,f1_score,confusion_matrix
from sklearn.model_selection import StratifiedKFold

import time


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
os.chdir('/content/gdrive/My Drive/santander_transcation_prediction')

In [4]:
col=[f'var_{i}' for i in range(200)]

In [6]:
df=pd.read_csv("train.csv.zip")

In [5]:
#Loading save files from drive
FE=pickle.load(open('STP_Dumping/Feature_engineering.pkl','rb'))
models=pickle.load(open('STP_Dumping/models.pkl','rb'))

### Get Prediction

In [16]:
def get_prediction(x):  #shape of x=(1,d) ,where d=number of raw features

  # freq encoding for each feature
  x_freq=np.zeros(x.shape[1])   #This will be used to store newly generated features
  i=0
  for c in col:
    x_=x[0,i]             # Getting value at feature_c
    try:
      x_freq[i]=FE[c][x_] # getting frequency of the value present at feature_c
    except:
      x_freq[i]=1         # If any 'value' that is not present in train data for that feature then give it frequency =1
    #x_freq[i]=pd.Series(x[0,i]).map(FE[c]).fillna(1)
    i+=1

  x=np.concatenate((x,x_freq.reshape(1,-1)),axis=1)  # orignal and new features
  
  #Predictions
  pred=np.zeros(len(x))
  for clf in models.values():    #for each trained lgbm classifier
    pred+=clf.predict(x)        #returns probablity value
  final_prediction=pred/len(models) #averaging predictions fro different lgbm classifier
  y_pred_label=np.where(final_prediction<0.5,0,1)  #Getting label from probablity value

  return int(y_pred_label)




In [32]:
start=time.time()
print("Predicted value :",get_prediction(df[col][0:1].values))
print("Original value  :",int(df['target'][0:1].values))
print("Time  Elapsed   :",time.time()-start)

Predicted value : 0
Original value  : 0
Time  Elapsed   : 0.11082243919372559


In [35]:
start=time.time()
print("Predicted value :",get_prediction(df[col][10:11].values))
print("Original value  :",int(df['target'][10:11].values))
print("Time  Elapsed   :",time.time()-start)

Predicted value : 0
Original value  : 0
Time  Elapsed   : 0.11700153350830078


In [36]:
start=time.time()
print("Predicted value :",get_prediction(df[col][13:14].values))
print("Original value  :",int(df['target'][13:14].values))
print("Time  Elapsed   :",time.time()-start)

Predicted value : 1
Original value  : 1
Time  Elapsed   : 0.10606002807617188


### Get Perfomance

In [27]:
def Get_Perfomance(x,y):   # x should be ndarray , y can be "pandas.core.series.Series" or "ndarray"

  # freq encoding for each feature
  x_freq=np.zeros(x.shape)   #This will be used to store newly generated features
  i=0
  for c in col:
    x_freq[:,i]=pd.Series(x[:,i]).map(FE[c]).fillna(1).values
    i+=1

  #concatation
  x=np.concatenate((x,x_freq),axis=1)  # orignal and new features

  #Predictions
  pred=np.zeros(len(x))
  for clf in tqdm(models.values()):    #for each trained lgbm classifier
    pred+=clf.predict(x)        #returns probablity value
  final_prediction=pred/len(models) #averaging predictions fro different lgbm classifier
  #y_pred_label=np.where(final_prediction<0.5,0,1)  #Getting label from probablity value

  
  auc_score=roc_auc_score(y,final_prediction)

  return auc_score




In [28]:
print("AUC score :",Get_Perfomance(df[col].values,df['target']))

100%|██████████| 10/10 [28:00<00:00, 168.06s/it]

AUC score : 0.9401459391211611
